In [14]:
import lilac as ll

items = [
  {'questions': ['A', 'B']},
  {'questions': ['C']},
  {'questions': ['D', 'E']},
]
config = ll.DatasetConfig(
  namespace='local',
  name='tutorial2',
  source=ll.DictSource(items),
)
dataset = ll.create_dataset(config, overwrite=True)
dataset.to_pandas()


Reading from source dict...: 100%|██████████| 3/3 [00:00<00:00, 837.97it/s]

Dataset "tutorial2" written to ./datasets/local/tutorial2


,questions.*
0,"[A, B]"
1,[C]
2,"[D, E]"


In [16]:
def add_prefix(question):
  return 'Q: ' + question


res = dataset.map(
  add_prefix, input_path=('questions', '*'), output_column='questions_with_prefix', overwrite=True
)

print(list(res))

dataset.to_pandas()


Scheduling task "0dbe18d9b6ed460c92b347258bd12344": "[local/tutorial2][1 shards] map "add_prefix" to "questions_with_prefix"".
Wrote map output to ./datasets/local/tutorial2/questions_with_prefix-00000-of-00001.parquet
[]


[Shard 0/1] map "add_prefix" to "('questions_with_prefix',)": 100%|██████████| 3/3 [00:00<00:00, 420.47it/s]


,questions.*,questions_with_prefix.*
0,"[A, B]","[Q: A, Q: B]"
1,[C],[Q: C]
2,"[D, E]","[Q: D, Q: E]"


Task finished "0dbe18d9b6ed460c92b347258bd12344": "[local/tutorial2][1 shards] map "add_prefix" to "questions_with_prefix"" in 4s.


In [10]:
import lilac as ll

items = [
  {
    'text': 'Apple Inc. is an American multinational technology company headquartered in Cupertino, California, (Company: Apple Inc.) that designs, develops, and sells consumer electronics, computer software, and online services.'
  },
  {
    'text': 'Google LLC is an American multinational technology company (Company: Google LLC) that specializes in Internet-related services and products, which include online advertising technologies, a search engine, cloud computing, software, and hardware.'
  },
]
config = ll.DatasetConfig(
  namespace='local',
  name='tutorial3',
  source=ll.DictSource(items),
)
dataset = ll.create_dataset(config, overwrite=True)
dataset.to_pandas()


def extract_company_with_coordinates(text):
  import re

  pattern = r'\(Company: ([^\)]+)\)'
  matches = re.finditer(pattern, text)
  return [ll.span(m.start(), m.end(), {'Company': m.group(1)}) for m in matches]


dataset.map(
  extract_company_with_coordinates, input_path='text', output_column='company', overwrite=True
)


Reading from source dict...: 100%|██████████| 2/2 [00:00<00:00, 1251.66it/s]

Dataset "tutorial3" written to ./datasets/local/tutorial3
Scheduling task "22c95a8997ef490d8a28e56f9933cf94": "[local/tutorial3][1 shards] map "extract_company_with_coordinates" to "company"".
Wrote map output to ./datasets/local/tutorial3/company-00000-of-00001.parquet


[Shard 0/1] map "extract_company_with_coordinates" to "('company',)": 100%|██████████| 2/2 [00:00<00:00, 849.65it/s]


In [21]:
df = dataset.to_pandas()
print(df.columns)


Index(['text', 'company.*', 'company.*.Company'], dtype='object')


In [18]:
df[:1]['company.*.Company']


0    [Apple Inc.]
Name: company.*.Company, dtype: object

In [ ]:
ll.start_server()


INFO:     Started server process [65094]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5432 (Press CTRL+C to quit)


In [ ]:
ll.start_server()


INFO:     Started server process [65547]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5432 (Press CTRL+C to quit)


In [2]:
import lilac as ll

config = ll.DatasetConfig(
  namespace='local',
  name='companies',
  source=ll.HuggingFaceSource(dataset_name='atmallen/companies_azaria_mitchell'),
)
dataset = ll.create_dataset(config, overwrite=True)
dataset.to_pandas()


/Users/dsmilkov/code/lilac/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset "companies" written to ./data/datasets/local/companies


,statement,label,__hfsplit__,__rowid__,overview.sbert.*,overview.openai.*,overview.sbert.*.embedding,overview.openai.*.embedding
0,Shell operates in the industry of oil & gas op...,true,train,a151a69e-3efe-4fa8-a0a9-302f8c9a9fe0,None,None,None,None
1,Nestle operates in the industry of Banking.,false,train,cd9f8dd0-3ff0-4d45-a8ff-39bcd6a87e9d,None,None,None,None
2,Itau Unibanco Holding operates in the industry...,true,train,602ceff5-1973-44fd-911c-fc83a1096948,None,None,None,None
3,Raytheon Technologies operates in the industry...,true,train,7990194a-6e50-45b5-97e6-bc2793b95ede,None,None,None,None
4,Manulife operates in the industry of insurance.,true,train,8de5f7d2-9d63-42cd-bb64-19f911357166,None,None,None,None
...,...,...,...,...,...,...,...,...
1195,Alphabet has headquarters in South Korea.,false,test,aee1baa1-79d5-49cc-8a7a-45199421872a,None,None,None,None
1196,"Iberdrola is a holding company, which engages ...",false,test,0e31deea-2000-46ce-9579-50991f953257,None,None,None,None
1197,Dow has headquarters in Italy.,false,test,b72e5655-7933-4282-b743-4e405defc58f,None,None,None,None
1198,Merck & Co. engages in the provision of health...,true,test,6b39b493-1857-47df-9485-55b21cb9bd1c,None,None,None,None


In [ ]:
ll.start_server()


INFO:     Started server process [69441]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5432 (Press CTRL+C to quit)
[Shard 0/1] map "extract_company" to "('company',)":  53%|█████▎    | 636/1200 [18:29<06:22,  1.47it/s]/var/folders/zn/brkm752x5673zb89tjjqzzs00000gn/T/ipykernel_69441/3762146513.py:33: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
[Shard 0/1] map "extract_company" to "('company',)":  67%|██████▋   | 803/1200 [20:44<06:13,  1.06it/s]/it]

In [4]:
import instructor
from openai import OpenAI
from pydantic import BaseModel, Field

# Enables `response_model`
# client = instructor.patch(OpenAI())
import lilac as ll


dataset = ll.get_dataset('local', 'companies')

if 'get_ipython' in globals():
  del globals()['get_ipython']
if 'exit' in globals():
  del globals()['exit']
if 'quit' in globals():
  del globals()['quit']


class CompanyDetail(BaseModel):
  """Details of a company."""

  name: str = Field(description='Company name')
  age: int = Field(description='The estimated age of the company')


def extract_company(text):
  # client = instructor.patch(OpenAI())
  # company = client.chat.completions.create(
  #   model='gpt-3.5-turbo',
  #   response_model=CompanyDetail,
  #   messages=[
  #     {'role': 'user', 'content': text},
  #   ],
  # )
  # return company.model_dump()
  return None


dataset.map(extract_company, input_path='statement', output_column='company')


Scheduling task "8606711d2a0a4ed99ad59bddd2697f07": "[local/companies][1 shards] map "extract_company" to "company"".


/var/folders/zn/brkm752x5673zb89tjjqzzs00000gn/T/ipykernel_69441/3762146513.py:33: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
[Shard 0/1] map "extract_company" to "('company',)":  50%|████▉     | 594/1200 [17:48<07:53,  1.28it/s]/var/folders/zn/brkm752x5673zb89tjjqzzs00000gn/T/ipykernel_69441/3762146513.py:33: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
[Shard 0/1] map "extract_company" to "('company',)":  50%|████▉     | 595/1200 [17:48<07:59,  1.26it/s]/var/folders/zn/brkm752x5673zb89tjjqzzs00000gn/T/ipykernel_69441/3762146513.py:33: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 t

Wrote map output to ./data/datasets/local/companies/company-00000-of-00001.parquet


[Shard 0/1] map "extract_company" to "('company',)": 1201it [00:00, 13499.72it/s]              


Task finished "8606711d2a0a4ed99ad59bddd2697f07": "[local/companies][1 shards] map "extract_company" to "company"" in 6s.


/var/folders/zn/brkm752x5673zb89tjjqzzs00000gn/T/ipykernel_69441/3762146513.py:33: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
[Shard 0/1] map "extract_company" to "('company',)":  50%|█████     | 602/1200 [17:54<07:24,  1.35it/s]/var/folders/zn/brkm752x5673zb89tjjqzzs00000gn/T/ipykernel_69441/3762146513.py:33: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
[Shard 0/1] map "extract_company" to "('company',)":  50%|█████     | 603/1200 [17:54<07:02,  1.41it/s]/var/folders/zn/brkm752x5673zb89tjjqzzs00000gn/T/ipykernel_69441/3762146513.py:33: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 t